In [89]:
import geopandas as gpd
import math
from shapely.geometry import Point, LineString, Polygon, MultiPoint, MultiLineString, MultiPolygon

In [90]:
buildings= gpd.read_file('../tests/data/buildings.geojson')
road= gpd.read_file('../tests/data/road.geojson')

In [91]:
# for d in buildings.columns:
#     print(d)
print(buildings['M_R_Code_2'])
# print(road)

0       TW611
1       TW611
2       TW611
3       TW611
4       TW611
        ...  
3244    TW802
3245    TW761
3246    TW761
3247    TW761
3248    TW763
Name: M_R_Code_2, Length: 3249, dtype: object


In [92]:
class bdcPoint:
    	
	def __init__(self):
		
		self.x = 0
		self.y = 0

# Constant integers for directions
RIGHT = 1
LEFT = -1
ZERO = 0

def directionOfPoint(A, B, P):
    """[Provides direction of the point in respect to line]

    Args:
        A ([type]): [Starting Point of Line]
        B ([type]): [Ending Point of Line]
        P ([type]): [Point of Interest]

    Returns:
        [Direction(int)]: [Left,Right and Zero : -1,1,0 ( Left direction, Right Direction , On Line)]
    """
    global RIGHT, LEFT, ZERO
	
	# Subtracting co-ordinates of
	# point A from B and P, to
	# make A as origin
 
    B.x -= A.x
    B.y -= A.y
    P.x -= A.x
    P.y -= A.y

    # Determining cross Product
    """"The Cross-Product has an interesting Property which will be used to determine direction of a point from a line segment. That is, the cross-product of two points is positive if and only if the angle of those point at origin (0, 0) is in counter-clockwise. And conversely the cross-product is negative if and only if the angle of those point at origin is in clockwise direction."""
    cross_product = B.x * P.y - B.y * P.x

    # Return RIGHT if cross product is positive
    if (cross_product > 0):
        return RIGHT
        
    # Return LEFT if cross product is negative
    if (cross_product < 0):
        return LEFT

    # Return ZERO if cross product is zero
    return ZERO

def addDirectionLength(direction,length,row):
    #right-even, left-odd number 
    print(length)
    if (direction == 1):
        print("Right Direction")
        row["road_direction"] = "right"
        row["road_length_d_code"] = round_up_to_even(length)
        
    elif (direction == -1):
        print("Left Direction")
        row["road_direction"] = "left"
        row["road_length_d_code"] = round_up_to_odd(length)
    else:
        row["road_direction"] = "on_line"
        
def getStartEndLength(line):
    line_bound = line.geometry.boundary
    df_line=line_bound.explode(index_parts=True)
    start_point,end_point=df_line.iloc[0],df_line.iloc[1]
    projected_line = line.to_crs(epsg=32644)
    length= projected_line.geometry.length
    float_length=length.tolist()
    return start_point,end_point,float_length[0]


def round_up_to_even(f):
    return math.ceil(f / 2.) * 2

def round_up_to_odd(f):
    return math.ceil(f) // 2 * 2 + 1


In [94]:

for index, row in buildings.iterrows():
    print (index)
    # print (row)
    road_code = row.M_R_Code_2
    poi = row.geometry
    road_linked_wrt_point= road.loc[road['M_R_Code_2'] == road_code]
    start_point,end_point,length=getStartEndLength(road_linked_wrt_point)
    start = bdcPoint()
    end = bdcPoint()
    point = bdcPoint()
    start.x,start.y,end.x,end.y,point.x,point.y=start_point.x,start_point.y,end_point.x,end_point.y,poi.x,poi.y
    direction=directionOfPoint(start,end,point)
    addDirectionLength(direction,length,row)
    print(row)
    break

0
Left Direction
OBJECTID_1                                                            1
Join_Count                                                            1
TARGET_FID                                                            0
Id                                                                    0
Join_Cou_1                                                            1
TARGET_F_1                                                            0
JOIN_FID                                                            110
OBJECTID                                                            116
Ward_No                                                             0.0
M_R_Code                                                          TW611
Width                                                              20.0
ROW                                                                36.0
Class                                                                 C
Priority                                       

In [ ]:

line_file = line_file.to_crs(epsg=32645)
line_file['length']= line_file.geometry.length